In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy

import torch as t
from torch.optim import Adam

from modeling.chessformer import ChessFormerA2C
from reinforcement.actor_critic.a2c import A2CTrainer
from reinforcement.players import A2CModelPlayer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAX_BATCH_SIZE = 64
EXPERIMENT_NAME = 'a2c_bigger_lr5e-6_history_len_1_vs_opponent_1'

In [13]:
model = ChessFormerA2C(nb_encoder_layers=6, nb_decoder_layers_policy=3, embedding_dim=512,
                       bottleneck_hidden_dim=2048, dim_per_head=64, nb_head=8).to('cuda')

optimizer = Adam(model.parameters(), lr=5e-6)
#competitor = DummyPlayer()
competitor = A2CModelPlayer(deepcopy(model), model_device='cuda')

In [ ]:
trainer = A2CTrainer(model,
                     optimizer,
                     competitor,
                     MAX_BATCH_SIZE,
                     EXPERIMENT_NAME,
                     model_device='cuda',
                     history_len=1)

2023-12-07 07:44:18.688300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 07:44:19.509806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-07 07:44:19.509887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-07 07:44:19.509891: W 

In [ ]:
trainer.train(100_000)

 44%|████▍     | 43939/100000 [5:31:54<6:52:25,  2.27it/s] 

In [ ]:
t.save(model.state_dict(), f'runs/{EXPERIMENT_NAME}/model.pt')